<a href="https://colab.research.google.com/github/sumankanukollu/denseDepthMaps_S15/blob/master/scriptToGenerate_FG_transparent_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Mount drive

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# 2. Import Fully-Covolutional Network model with a ResNet-101

In [0]:
from torchvision import models
fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()

In [0]:
import sys,os
from pathlib import Path


dataSet = Path('/content/drive/My Drive/EVA4/S15A_RCNN/original_rgb_imgs')
sys.path.append(str(dataSet))

print(sys.path)
os.chdir(dataSet)
!ls

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T

print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
#img = Image.open('10_suman.png')
#plt.imshow(img); plt.show()

# 3. To generate masks for the FG Images define the helper functions 

In [0]:
from IPython.core.debugger import  set_trace
import numpy as np
def decode_segmap(image,name, nc=21, imgclr='black'):
  
  label_colors_rgb = np.array([(0,0,0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])
  
  label_colors_black = np.array([(255,255,255),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)])
  
  label_colors_white = np.array([(0,0,0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (255,255,255), (255,255,255),(255,255,255),(255,255,255),(255,255,255),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (255,255,255), (255,255,255),(255,255,255),(255,255,255),(255,255,255),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (255,255,255),(255,255,255),(255,255,255),(255,255,255),(255,255,255),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (255,255,255),(255,255,255),(255,255,255),(255,255,255),(255,255,255)])
  if imgclr=='black':
    label_colors = label_colors_black
  elif imgclr=='white':
    label_colors = label_colors_white
  else:
    label_colors = label_colors_rgb

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
    
  
  rgb = np.stack([r, g, b], axis=2)
  im = Image.fromarray(rgb)
  maskpath = os.path.join(dataSet,'mask',name+'_mask.png')
  print('Image mask is saved at : {}'.format(maskpath))
  im.save(maskpath)
  return rgb

In [0]:
def segment(net, path, show_orig=True, dev='cuda',imgclr='white'):
  img = Image.open(path)

  if show_orig: plt.imshow(img); plt.axis('off'); plt.show()
  # Comment the Resize and CenterCrop for better inference results
  trf = T.Compose([T.Resize(128), 
                   #T.CenterCrop(224), 
                   #T.CenterCrop(224), 
                   #T.Grayscale(num_output_channels=1),
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
  inp = trf(img).unsqueeze(0).to(dev)
  out = net.to(dev)(inp)['out']
  om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
  rgb = decode_segmap(om,name=path.split('.')[0],imgclr=imgclr)
  plt.imshow(rgb); plt.axis('off'); plt.show()

In [0]:
os.chdir(os.path.join(dataSet,'fg_96'))
!pwd
for img in os.listdir('.'):
    #print(img)
    if img.endswith('.jpg'):
      segment(fcn, img,imgclr='white')
      

# 4. Resize images

In [0]:
from PIL import Image

In [0]:
def convert2JpgNresz(w,h):
	basewidth = w
	hsize =  h
	start =1
	for im in os.listdir('.'):
		if im.endswith('.png'):
			#os.remove(im)
			img = Image.open(im)
			#wpercent = (basewidth/float(img.size[0]))
			#hsize = int((float(img.size[1])*float(wpercent)))
			img = img.resize((basewidth,hsize), Image.ANTIALIAS)
			newName = im.split('.png')[0]+'.jpg'	
			img.save(newName)
			
			start +=1

In [0]:
homepath = Path('/content/drive/My Drive/pytorch_Framework_suman/RCNN_S14_ImageSegmentation/original_jpg_imgs/fg/')
os.chdir(homepath)
convert2JpgNresz(90,90)

# 5. Convert to gray scale and verify image sizes

In [0]:
def checkSz():
    for i in os.listdir('.'):
        im = Image.open(i)
        print(im.size)


def convertRgbToGrayScaleImg():
    for i in os.listdir('.'):
        if i.endswith('.jpg'):
            im = Image.open(i).convert('L')
            im.save(i)
homepath = Path('/content/drive/My Drive/pytorch_Framework_suman/RCNN_S14_ImageSegmentation/original_jpg_imgs/mask/')
os.chdir(homepath)
convertRgbToGrayScaleImg()